In [1]:
import numpy as np
import pandas as pd
import gc

C:\Users\zhengzong\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
chunksize = 100000 * 400
df = pd.read_csv(r"D:\BaiduNetdiskDownload\201803.csv", chunksize=chunksize, iterator=True) ### 这里切换 年份的数据集

In [3]:
a = df.get_chunk()

In [4]:
df_chunk = a
df_chunk

,出站时间,到达车站编号,卡号,起始车站编号,进站时间,卡大类（99一卡通，1一票卡）,卡的子类型,扣费金额（100倍数，1是员工票或者计次票，0是免费票福利票车站工作票）,卡周期,对账日期,结算日期,安全密钥,闸机号,运营单位,钱包余额,交易序列号
0,2018030112:13:28,150995224,43891592,150996029,2018030111:23:00,1,3,500.0,9,2018030100:00:00,2018030200:00:00,68248,520165380,41,0,1
1,2018030112:12:55,150995224,113161522,150996279,2018030111:41:00,1,3,400.0,8,2018030100:00:00,2018030200:00:00,68248,520165380,41,0,1
2,2018030112:12:50,150995224,106779752,150996279,2018030111:41:00,1,3,400.0,62,2018030100:00:00,2018030200:00:00,68248,520165380,41,0,1
3,2018030112:09:40,150995224,44132629,150996525,2018030111:30:00,1,3,300.0,43,2018030100:00:00,2018030200:00:00,68248,520165380,41,0,1
4,2018030112:09:14,150995224,57346980,150995457,2018030111:25:00,1,3,500.0,147,2018030100:00:00,2018030200:00:00,68248,520165380,41,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39999995,2018030923:24:30,151019291,19695798,150996533,2018030921:53:00,99,1,560.0,0,2018030900:00:00,2018030900:00:00,59000166,526326594,75,11800,876
39999996,2018030923:30:53,150995203,60244737,151019779,2018030922:19:00,99,1,600.0,0,2018030900:00:00,2018030900:00:00,19000013,520160012,41,2600,498
39999997,2018030923:24:25,150995203,46233582,150996285,2018030922:22:00,99,1,600.0,0,2018030900:00:00,2018030900:00:00,19000013,520160012,41,1790,2738
39999998,2018030923:31:32,150995203,60452569,150995466,2018030922:39:00,99,1,600.0,0,2018030900:00:00,2018030900:00:00,19000007,520160006,41,2950,80


In [5]:
df_chunk['出站时间'] = pd.to_datetime(df_chunk['出站时间'],format="%Y%m%d%H:%M:%S")
df_chunk['进站时间'] = pd.to_datetime(df_chunk['进站时间'],format="%Y%m%d%H:%M:%S")

In [6]:
df_chunk['日期'] = df_chunk['出站时间'].apply(lambda x:str(x.year)+'-'+str(x.month)+'-'+str(x.day))

In [7]:
a = pd.DataFrame(df_chunk.groupby(['卡号'])[['日期','钱包余额']].nunique())

In [8]:
a = a.reset_index()
a

,卡号,日期,钱包余额
0,1082,2,4
1,1123,1,2
2,1165,1,1
3,1266,1,2
4,1691,1,1
...,...,...,...
9920073,901000112934,2,4
9920074,901000113544,1,1
9920075,901000115481,1,1
9920076,901000118188,1,3


### 聚类

In [9]:
from sklearn.cluster import KMeans 

In [10]:
X = a.copy()
# X['日期_'] = X[['日期']
X = X[['日期','钱包余额']].values

In [11]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()

# scaler = scaler.fit(X)  # 本质生成 max(x) 和 min(x)
# X = scaler.transform(X)

In [12]:
#Model Build
kmeansmodel = KMeans(n_clusters= 2, init='k-means++', random_state=0)
y_kmeans = kmeansmodel.fit_predict(X)

In [13]:
a['聚类结果'] = y_kmeans

In [14]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"] #设置字体
plt.rcParams["axes.unicode_minus"]=False #该语句解决图像中的“-”负号的乱码问题

plt.scatter(X[y_kmeans == 0, 0], X[y_kmeans == 0, 1], s = 10, c = 'red', label = 'Cluster 0')
plt.scatter(X[y_kmeans == 1, 0], X[y_kmeans == 1, 1], s = 10, c = 'blue', label = 'Cluster 1')

#plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 12], s = 300, c = 'yellow', label = 'Centroids')
plt.title('Clusters result')
plt.xlabel('活跃天数')
plt.ylabel('钱包余额变化')
plt.legend()


plt.savefig('./18Kmeans1.png', dpi = 400)
plt.show()

<Figure size 640x480 with 1 Axes>

In [15]:
a.columns = ['卡号','活跃天数','钱包余额变化','聚类结果']
a.to_csv("./18年活跃天数聚类_两指标.csv",index=False,encoding='utf_8_sig')

#### 活跃用户的百分比 

In [19]:
a.groupby(['聚类结果'])['活跃天数'].mean()

聚类结果
0    1.613869
1    5.168298
Name: 活跃天数, dtype: float64

可以看出类别1为活跃人群

In [20]:
a['聚类结果'].value_counts()

0    7706602
1    2213476
Name: 聚类结果, dtype: int64

In [21]:
2213476 / ( 2213476 +  7706602) 

0.22313090683359546

活跃用户的百分比为22.31%